# Disaggregation of appliances from generated load profiles

In [43]:
# General libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.optimizers import Adam

%matplotlib inline

In [44]:
df_import = pd.read_csv("data/14_days/DeviceProfiles.HH1.Electricity.csv", delimiter=";", parse_dates=["Time"])
df_import.head()

,Electricity.Timestep,Time,"HH1 - Living room - Living Room Light (Energy Saving Lamp, 20W) [kWh]",HH1 - Kitchen - Kitchen Light (20W) [kWh],HH1 - Bath - Bathroom Light (20W) [kWh],HH1 - Bath - Bathroom Mirror Light 10 W (LED) [kWh],HH1 - Bedroom - Bedroom Light (20W) [kWh],HH1 - Children's room - Children Room Light Device (20W) [kWh],HH1 - Kitchen - AFK BM-2N [kWh],HH1 - Kitchen - Egg Cooker / Russell Hobbs 14048-56 Stylo [kWh],...,HH1 - Living room - TV Medion MD20123_DE_A [kWh].1,HH1 - Living room - CD/DVD Player / Philips DVDR 725 H [kWh].1,HH1 - Living room - Home Server 50 W [kWh],HH1 - Bath - Electric Razor Braun Cruzer 5 [kWh],HH1 - Living room - Laptop Sony Vaio SVE151G11M [kWh].1,HH1 - Living room - Router O2 Box 6431 [kWh],HH1 - Children's room - TV Medion MD20123_DE_A [kWh].1,HH1 - Children's room - Sony PlayStation 3 [kWh].1,HH1 - Kitchen - Vacuum Cleaner Robot / iRobot Roomba 555 [kWh].1,"HH1 - Vehicle Pool - Car 2, 22kW Charging Power, avg. Speed 30 km/h [kWh]"
0,0,2021-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.378580e-08,2.646191e-07,0.000013,2.629474e-07,7.422109e-07,0.000002,8.639636e-08,0.000001,7.979168e-07,0
1,1,2021-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.611321e-08,2.971380e-07,0.000014,2.629474e-07,8.046940e-07,0.000002,8.432451e-08,0.000001,9.279561e-07,0
2,2,2021-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.314018e-08,3.220455e-07,0.000011,2.629474e-07,8.162093e-07,0.000002,8.069800e-08,0.000001,9.457638e-07,0
3,3,2021-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.849064e-08,3.476178e-07,0.000016,2.643636e-07,7.960612e-07,0.000002,7.724810e-08,0.000001,9.313386e-07,0
4,4,2021-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.637703e-08,2.740711e-07,0.000014,2.643636e-07,7.645697e-07,0.000002,8.092908e-08,0.000001,7.991072e-07,0


In [45]:
appliance_columns = [
    "HH1 - Kitchen - Single Stove Plate [kWh]",
    "HH1 - Kitchen - Nespresso Coffee Machine, Single Cup [kWh]",
    "HH1 - Kitchen - Microwave / Panasonic NN 5259 [kWh]",
    "HH1 - Kitchen - Dishwasher NEFF SD6P1F (2011) [kWh]",
    "HH1 - Kitchen - Washing Machine / Bosch WAE 28143 [kWh]"
]

In [46]:
df_selected = df_import[["Electricity.Timestep",
    "Time"] + appliance_columns].copy()
#del(df_import)

In [47]:
def count_non_zero_rows(dataframe, column):
    return len(dataframe) - dataframe[column].isin([0]).sum()

In [48]:
count_non_zero_rows(df_selected, "HH1 - Kitchen - Washing Machine / Bosch WAE 28143 [kWh]")

46354

In [49]:
target_names = ["Stove", "Coffee_machine", "Microwave", "Dishwasher", "Washing_machine"]
name_mappings = dict(zip(appliance_columns, target_names))
name_mappings

{'HH1 - Kitchen - Single Stove Plate [kWh]': 'Stove',
 'HH1 - Kitchen - Nespresso Coffee Machine, Single Cup [kWh]': 'Coffee_machine',
 'HH1 - Kitchen - Microwave / Panasonic NN 5259 [kWh]': 'Microwave',
 'HH1 - Kitchen - Dishwasher NEFF SD6P1F (2011) [kWh]': 'Dishwasher',
 'HH1 - Kitchen - Washing Machine / Bosch WAE 28143 [kWh]': 'Washing_machine'}

In [50]:
df = df_selected.rename(columns=name_mappings).copy()
#del(df_selected)

In [51]:
df.head(61)

,Electricity.Timestep,Time,Stove,Coffee_machine,Microwave,Dishwasher,Washing_machine
0,0,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
1,1,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
2,2,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
3,3,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
4,4,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
56,56,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
57,57,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
58,58,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
59,59,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0


## Create target column

In [52]:
df["Target"] = df[list(name_mappings.values())].sum(axis=1)
df

,Electricity.Timestep,Time,Stove,Coffee_machine,Microwave,Dishwasher,Washing_machine,Target
0,0,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
1123195,1123195,2021-01-13 23:59:00,0.0,0.0,0.0,0.0,0.0,0.0
1123196,1123196,2021-01-13 23:59:00,0.0,0.0,0.0,0.0,0.0,0.0
1123197,1123197,2021-01-13 23:59:00,0.0,0.0,0.0,0.0,0.0,0.0
1123198,1123198,2021-01-13 23:59:00,0.0,0.0,0.0,0.0,0.0,0.0


## Fix timestamp seconds

In [53]:
df["Time"] = [dt.replace(second=(df["Electricity.Timestep"][i]%60)) for i,dt in enumerate(df["Time"])]
df

,Electricity.Timestep,Time,Stove,Coffee_machine,Microwave,Dishwasher,Washing_machine,Target
0,0,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2021-01-01 00:00:01,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2021-01-01 00:00:02,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2021-01-01 00:00:03,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2021-01-01 00:00:04,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
1123195,1123195,2021-01-13 23:59:55,0.0,0.0,0.0,0.0,0.0,0.0
1123196,1123196,2021-01-13 23:59:56,0.0,0.0,0.0,0.0,0.0,0.0
1123197,1123197,2021-01-13 23:59:57,0.0,0.0,0.0,0.0,0.0,0.0
1123198,1123198,2021-01-13 23:59:58,0.0,0.0,0.0,0.0,0.0,0.0


## Export data for further preprocessing

In [14]:
df.to_csv("data/14_days/data_reduced.csv")